# Лабораторная работа по 3 модели

## Команда __Марковцы:__
- Меркулов Александр
- Калугин Евгений
- Фомин Дмитрий
- Чукаев Иван

## Описание модели

На вход $n$-канальной СМО поступает простейший поток заявок с интенсивностью $\lambda$.

Интенсивность простейшего потока обслуживания каждого канала $\mu$.

Если заявка застаёт все каналы свободными, она принимается на обслуживание и обслуживается любым одним из $n$-каналов.

Если заявка застаёт свободным хотя бы один канал, то она принимается на обслуживание любым из свободных каналов и обслуживается до конца.

Если заявка застаёт все каналы занятыми, то она получает отказ (покидает систему не обслуженной).

После окончания обслуживания одной заявки освобождается один канал.

Состояние рассмотренной системы будем связывать с числом заявок, находящихся в системе.

## Постановка задачи

В городе имеется n станций техобслуживания.

Сервисы обслуживает $\mu$ машин в день, интенсивность входного потока заявок - $\lambda$ машин в день. 

После окончания обслуживания один сервис освобождается. 

Если вновь прибывшая машина застаёт свободным хотя бы один сервис, то она принимается на обслуживание одним из свободных сервисов. 

Если нет свободных сервисов в городе, то машина уезжает на обслуживание в другой город.

## Граф состояний

![image.png](./images/graph.jpg)

## Мат. модель.



0\) $\;\mu\Pi_1=\lambda\Pi_0$

1\) $\;\lambda\Pi_0+2\mu\Pi_2=(\lambda+\mu)\Pi_1$

2\) $\;\lambda\Pi_1+3\mu\Pi_3=(\lambda+2\mu)\Pi_2$

…

k) $\;\lambda\Pi_{k-1}+(k+1)\mu\Pi_{k+1}=(\lambda+k\mu)\Pi_k$

…

n) $\;\lambda\Pi_{n-1}= n\mu\Pi_n$ <br><br><hr><br>

1\) $\;\Pi_1=\frac{\lambda}{\mu}\Pi_0$<br><br>

2\) $\;2\mu\Pi_2=(\lambda+\mu)\Pi_1-\lambda\Pi_0$ <br><br>&nbsp;&nbsp;&nbsp;
    $\;2\mu\Pi_2=(\lambda+\mu)\frac{\lambda}{\mu}\Pi_0-\lambda\Pi_0$ <br><br>&nbsp;&nbsp;&nbsp;
    $\;2\mu\Pi_2=\frac{\lambda^2+\lambda\mu-\lambda\mu}{\mu}\Pi_0$ <br><br>&nbsp;&nbsp;&nbsp;
    $\;\Pi_2=\frac{1}{2}(\frac{\lambda}{\mu})^2\Pi_0$ <br>&nbsp;&nbsp;&nbsp;

3\) $\;3\mu\Pi_3=(\lambda+2\mu)\Pi_2-\lambda\Pi_1$ <br><br>&nbsp;&nbsp;&nbsp;
    $\;3\mu\Pi_3=(\lambda+2\mu)(\frac{1}{2})(\frac{\lambda}{\mu})^2\Pi_0-\lambda\frac{\lambda}{\mu}\Pi_0$ <br><br>&nbsp;&nbsp;&nbsp;
    $\;3\mu\Pi_3=(\frac{\lambda}{\mu})^2(\frac{1}{2}(\lambda+2\mu)\Pi_0-\mu\Pi_0)$ <br><br>&nbsp;&nbsp;&nbsp;
    $\;3\mu\Pi_3=(\frac{\lambda}{\mu})^2\cdot\frac{\lambda}{2}\Pi_0$ <br><br>&nbsp;&nbsp;&nbsp;
    $\;\Pi_3=(\frac{\lambda}{\mu})^3\cdot\frac{1}{2\cdot3}\Pi_0$ <br>

…

k)  $\;\Pi_k=(\frac{\lambda}{\mu})^k\cdot\frac{1}{k!}\Pi_0$ <br>

0\) $\;\displaystyle\Pi_0=\frac{1}{\sum_{k=1}^n\Pi_k}=\frac{1}{\sum_{k=1}^n\big(\frac{\lambda}{\mu}\big)^k\cdot\frac{1}{k!}}$

## Программа

### Импорты

In [19]:
import math
import numpy as np

## Функции 

In [20]:
def get_list_Pi(Lambda: float, Mu: float, n: int) -> list[float]:
    list_Pi = []
    for i in range(n + 1):
        list_Pi.append(((Lambda / Mu)**i) / math.factorial(i))
    
    return list_Pi

In [21]:
def get_Pi_0(list_Pi: list[float]) -> float:
    return 1/sum(list_Pi)

In [22]:
def get_mean_busy_services(list_Pi: list) -> float:
    sum = 0 
    for i in range(len(list_Pi)):
        sum += i * list_Pi[i]
        
    return sum

In [23]:
# Получение состояний системы в определенные моменты времени

def get_states(Lambda: float, Mu: float, n: int, interval: int) -> np.ndarray:
    cur_cars = 0
    states = [0]

    for _ in range(interval):
        if cur_cars == 0:
            if np.random.poisson(Lambda, 1) != [0]:
                cur_cars += 1 # принимаем машину

        elif 0 < cur_cars < n:
            if np.random.poisson(Lambda, 1) != [0]: 
                cur_cars += 1 # принимаем машину
            if np.random.poisson(Mu * cur_cars, 1) != [0]: 
                cur_cars -= 1 # обслуживаем машину

        elif cur_cars == n:
            if np.random.poisson(Mu * cur_cars, 1) != [0]:
                cur_cars -= 1 # обслуживаем машину

        states.append(cur_cars)
        
    return states

## Эксперименты

In [24]:
Lambda = 6
Mu = 5
n = 4
interval = 1000

list_Pi = get_list_Pi(Lambda, Mu, n)
Pi_0 = get_Pi_0(list_Pi)
distribution = [Pi_k * Pi_0 for Pi_k in list_Pi]
mean_busy_services = get_mean_busy_services(distribution)

time_Lambda = Lambda/24
time_Mu = Mu/24
states = get_states(time_Lambda, time_Mu, n, interval)
check_mean = np.mean(states)

print(f'Распределение: {[round(elem, 3) for elem in distribution]}')
print(f'Выборка: {states} ')
print(f'Среднее {check_mean} ')
print(f'Pi_0 = {round(Pi_0, 3)}, среднее количество занятых сервисов: {round(mean_busy_services, 3)}')

Распределение: [0.304, 0.364, 0.219, 0.087, 0.026]
Pi_0 = 0.304, среднее количество занятых сервисов: 1.169
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 